# Higher Order Moments Correction
This module implements the adjustment based CDF matching with homogeneous predictions from a polynomial regression model.

In [1]:
from set_up import *
from pybreaks.adjust_higher_order_moments import HigherOrderMoments
pd.options.display.max_rows=7
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# this cell loads either test data with properties as set by the user or real data and allows to introduce a break
def load_data(real=False, trend=0.01, bias=0.1, breaktime=datetime(2012,7,1), 
              size=(0.2, 1.1), resample=False):
    if real:
        CAN, REF = load_real_data(707393, breaktime, size, resample)
    else:
        CAN, REF = load_test_data(trend, bias, breaktime, size, resample, seed=1234)
    return CAN, REF

In [3]:
@interact(TREND=(0, 0.05, 0.01), BIAS=(-0.5, 0.5, 0.1), BREAK_ADD=(-0.4, 0.4, 0.05), 
          BREAK_MULT=(0.7, 1.3, 0.05), ALPHA=(0.1,1.,0.1), RESAMPLE=['NONE', 'INPUT', 'OUTPUT'])
def HOM(USE_REAL_DATA=False, TESTDATA_TREND=0.01, TESTDATA_BIAS=0.1, BREAK_ADD=0.2, 
        BREAK_MULT=1.1, ALPHA=0.6, RESAMPLE='NONE'):
    
    BREAKTIME=datetime(2012,7,1)
    size=(BREAK_ADD, BREAK_MULT)
    
    resample = True if RESAMPLE=='INPUT' else False
    CAN, REF = load_data(USE_REAL_DATA, TESTDATA_TREND, TESTDATA_BIAS, BREAKTIME, 
                         size, resample)
    
    hom = HigherOrderMoments(CAN, REF, BREAKTIME, regress_resample=None, 
        bias_corr_method='cdf_match', filter=('both', 5), adjust_group=0, poly_orders=[1,2], 
        select_by='R', cdf_types=None)
    
    can_adjusted = hom.adjust(CAN[:BREAKTIME], use_separate_cdf=False, alpha=ALPHA, 
                              from_bins=False) 
    
    
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,4)) 
    if RESAMPLE == 'OUTPUT':
        hom.df_original.resample('M').mean().plot(ax=ax1)
    else:
        hom.df_original.plot(ax=ax1)
    hom.plot_adjustments(ax=ax2)
    
    fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(10,4))
    hom.plot_models(axs=[ax1, ax2])
    hom.plot_cdf_compare()
    
    #hom.cdf_can_train.plot_pdf()
    #hom.cdf_can_train.plot_cdf()
    #hom._cdf_can_pred.plot_pdf()
    #hom._cdf_can_pred.plot_cdf()
    #hom._cdf_can_obs.plot_pdf()
    #hom._cdf_can_obs.plot_cdf()
    #hom.plot_cdf_compare_separate()
   

aW50ZXJhY3RpdmUoY2hpbGRyZW49KENoZWNrYm94KHZhbHVlPUZhbHNlLCBkZXNjcmlwdGlvbj11J1VTRV9SRUFMX0RBVEEnKSwgRmxvYXRTbGlkZXIodmFsdWU9MC4wMSwgZGVzY3JpcHRpb27igKY=
